<a href="https://colab.research.google.com/github/AnshumanAI/Steps/blob/main/Steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of packages

In [ ]:
!pip install PyMuPDF
!pip install frontend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import fitz
# Function to extract text from a PDF
def extract_text_from_pdf(file_path):
    pdf_document = fitz.open(file_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text()
    return text
#Used the books I had, they might be pirated idk
textbook_paths = [
    "/content/drive/MyDrive/Hands-On Machine Learning with Scikit-Lear - Aurelien Geron.pdf",
    "/content/drive/MyDrive/DeepLearningBook.pdf",
    "/content/drive/MyDrive/The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf"
]

# Extracting text from each textbook
textbook_texts = [extract_text_from_pdf(path) for path in textbook_paths]

Creation of Tree

In [ ]:
from collections import defaultdict
import re

# Definition of the structure of a node
class TreeNode:
    def __init__(self, identifier, content=""):
        self.identifier = identifier
        self.content = content
        self.children = []

#  Hierarchical tree-based index
def create_tree_index(text, title_pattern, section_pattern, subsection_pattern):
    root = TreeNode("Root")
    current_chapter = None
    current_section = None
    current_subsection = None

    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if re.match(title_pattern, line):
            current_chapter = TreeNode(line)
            root.children.append(current_chapter)
            current_section = None
            current_subsection = None
        elif re.match(section_pattern, line):
            if current_chapter is not None:
                current_section = TreeNode(line)
                current_chapter.children.append(current_section)
                current_subsection = None
            else:
                print(f"Warning: Section '{line}' found before any chapter. Skipping.")
        elif re.match(subsection_pattern, line):
            if current_section is not None:
                current_subsection = TreeNode(line)
                current_section.children.append(current_subsection)
            else:
                print(f"Warning: Subsection '{line}' found before any section. Skipping.")
        else:
            if current_subsection:
                current_subsection.content += line + " "
            elif current_section:
                current_section.content += line + " "
            elif current_chapter:
                current_chapter.content += line + " "
            else:
                print(f"Warning: Text found before any chapter, section, or subsection: '{line}'")

    return root

# Patterns to match chapters, sections, and subsections
title_pattern = r"^Chapter \d+: .*$"
section_pattern = r"^\d+\.\d+ .*$"
subsection_pattern = r"^\d+\.\d+\.\d+ .*$"

# Indexes for each textbook
tree_indexes = [create_tree_index(text, title_pattern, section_pattern, subsection_pattern) for text in textbook_texts]

Streaming output truncated to the last 5000 lines.


RankBM25 (uses BM25 algorithm), to rank documents.

In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder
import torch

# Tokenizer and model for DPR-based retrieval
tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

# Function to truncate texts to the maximum input length
def truncate_text(text, max_length=512):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > max_length:
        tokens = tokens[:max_length]
    return tokenizer.convert_tokens_to_string(tokens)

# Tokenization and embeddings for the content using DPR
def embed_texts(texts, max_length=512):
    truncated_texts = [truncate_text(text, max_length) for text in texts]
    inputs = tokenizer(truncated_texts, return_tensors='pt', truncation=True, padding=True, max_length=max_length)
    outputs = model(**inputs)
    embeddings = outputs.pooler_output
    return embeddings

# BM25 retrieval
def bm25_retrieve(query, corpus):
    if not corpus:
        return []
    bm25 = BM25Okapi([doc.split() for doc in corpus if doc.strip()])
    return bm25.get_top_n(query.split(), corpus, n=5)

# DPR-based retrieval
def dpr_retrieve(query, corpus):
    if not corpus:
        return []
    query_embedding = embed_texts([query])
    corpus_embeddings = embed_texts(corpus)
    similarities = torch.nn.functional.cosine_similarity(query_embedding, corpus_embeddings)
    top_indices = torch.argsort(similarities, descending=True)[:5]
    return [corpus[i] for i in top_indices]

# Hybrid retrieval combining BM25 and DPR
def hybrid_retrieve(query, corpus):
    bm25_results = bm25_retrieve(query, corpus)
    dpr_results = dpr_retrieve(query, bm25_results)
    return dpr_results

# Example corpus and query
corpus = [node.content for node in tree_indexes[0].children if node.content.strip()]  # Sample corpus from the first textbook's chapters
query = "What is Support Vector Machine?"

# Retrieval
retrieved_content = hybrid_retrieve(query, corpus)
print(retrieved_content)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['1. The fundamental idea behind Support Vector Machines is to fit the widest possible “street” between the classes. In other words, the goal is to have the largest possible margin between the decision boundary that separates the two classes and the training instances. When performing soft margin classification, the SVM searches for a compromise between perfectly separating the two classes and having the widest possible street (i.e., a few instances may end up on the street). Another key idea is to use kernels when training on nonlinear datasets. 2. After training an SVM, a support vector is any instance located on the “street” (see the previous answer), including its border. The decision boundary is entirely determined by the support vectors. Any instance that is not a support vector (i.e., off the street) has no influence whatsoever; you could remove them, add more instances, or move them around, and as long as they stay off the street they won’t affect the decision boundary. Computi

In [ ]:
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder, pipeline
from rank_bm25 import BM25Okapi
import torch

# Reusing the functions defined in Step 3

# RAG system
def rag_system(query, tree_indexes):
    all_retrieved_content = []
    for tree in tree_indexes:
        corpus = [node.content for node in tree.children if node.content.strip()]
        retrieved_content = hybrid_retrieve(query, corpus)
        all_retrieved_content.extend(retrieved_content)
    return all_retrieved_content

# Example query
query = "Explain the concept of Deep Learning."
response = rag_system(query, tree_indexes)
print(response)

['Networks tf.nn.elu(), Nonsaturating Activation Functions, TensorFlow Implementation-Tying Weights, Variational Autoencoders, Neural Network Policies, Policy Gradients tf.nn.embedding_lookup(), Word Embeddings tf.nn.in_top_k(), Construction Phase, Training a Sequence Classifier tf.nn.max_pool(), Pooling Layer-Pooling Layer tf.nn.relu(), Construction Phase, Training to Predict Time Series-Training to Predict Time Series, Training to Predict Time Series, Learning to Play Ms. Pac-Man Using Deep Q-Learning tf.nn.sigmoid_cross_entropy_with_logits(), TensorFlow Implementation, Generating Digits, Policy Gradients-Policy Gradients tf.nn.sparse_softmax_cross_entropy_with_logits(), Construction Phase- Construction Phase, Training a Sequence Classifier tf.one_hot(), Learning to Play Ms. Pac-Man Using Deep Q-Learning tf.PaddingFIFOQueue, PaddingFifoQueue tf.placeholder(), Feeding Data to the Training Algorithm-Feeding Data to the Training Algorithm, Chapter 9: Up and Running with TensorFlow tf.pl

In [ ]:
from transformers import pipeline

# Initialization of the QA pipeline
qa_pipeline = pipeline("question-answering")

# QA system
def answer_question(query, context):
    inputs = {
        'question': query,
        'context': context
    }
    result = qa_pipeline(inputs)
    return result['answer']

# Example query and response generation
query = "What's Difference between Linear Regression and Neural Network?."
retrieved_content = rag_system(query, tree_indexes)
context = " ".join(retrieved_content)
answer = answer_question(query, context)
print("Answer:", answer)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Answer: Internal Covariate Shift problem


In [ ]:

query2=" Activation functions? "
retrieved_content2 = rag_system(query2, tree_indexes)
context2 = " ".join(retrieved_content2)
answer2 = answer_question(query2, context2)
print("Answer:", answer2)

Answer: Nonsaturating
